# Recommendation System

In [1]:
# Imporing libraries

import pandas as pd
import numpy as np
from google.oauth2 import service_account

import fr_core_news_sm
nlp = fr_core_news_sm.load()
from spacy.lang.fr.stop_words import STOP_WORDS 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Reading dataset
TABLE = 'jedha_recommendation.user_view_videos_enriched_v2'
def query_db(query):
    return pd.io.gbq.read_gbq(query.format(table = TABLE), dialect="standard", project_id='salto-datalab-pid2')

The assumptions to determine if the program is liked by a profile:
* The program was seen for over an hour,
* The program was viewed at 75%,
* More than 5 episodes of the program have been seen.


Here are the details of the important fields:
* count_profile_like: number of profiles who liked the program,
* count_profile_not_like: number of profiles who did not like the program (the profile started to watch the program, but did not fulfill one of the criteria),
* count_profile_view: number of profiles having watched the program (excluding views less than 30 seconds),
* ratio_liked: ratio between count_profile_like and count_profile_not_like

In [3]:
DATASET = """
SELECT 
    program.program_id,
    program.title,
    tag_genre, 
    tag_sous_genre,
    ROUND(mean_age, 0) as mean_age,
    count_episodes,
    ROUND(MAX(program_total_duration) / count_episodes, 2) as mean_episode_duration,
    
    SUM(CASE WHEN profile_type = 'DEFAULT_ADULT' OR profile_type = 'STANDARD_ADULT' THEN 1 ELSE 0 END) as count_profile_adult,
    SUM(CASE WHEN profile_type = 'DEFAULT_KID' OR profile_type = 'STANDARD_KID' THEN 1 ELSE 0 END) as count_profile_kid,
    SUM(CASE WHEN profile_type = 'DEFAULT_HOME' THEN 1 ELSE 0 END) as count_profile_home,
    
    SUM(CASE WHEN gender = 'f' THEN 1 ELSE 0 END) as count_profile_f,
    SUM(CASE WHEN gender = 'm' THEN 1 ELSE 0 END) as count_profile_m,
    SUM(CASE WHEN count_viewed = 1 THEN 1 ELSE 0 END) as count_profile_like,
    SUM(CASE WHEN count_viewed = 0 THEN 1 ELSE 0 END) as count_profile_not_like,
    COUNT(count_viewed) as count_profile_view,
    ROUND(MAX(program_total_duration), 2) as program_seconds_duration, 
    ROUND(SUM(total_seconds_viewed), 2) as total_seconds_viewed, 
    ROUND(SUM(CASE WHEN count_viewed = 1 THEN total_seconds_viewed ELSE 0 END), 2) as seconds_viewed_liked,
    ROUND(SUM(CASE WHEN count_viewed = 0 THEN total_seconds_viewed ELSE 0 END), 2) as seconds_viewed_not_like,
    program_description
FROM (
    # Generating 'count_viewed' (= 1 if viewed for more than 1 hour or 3/4 of program duration)
    # Seconds view of each profile on a program (keeping only seconds viewed > 120s.)
    SELECT 
        pid,
        gender,
        profile_type,
        uv.program_id,
        AVG(count_episodes) as count_episodes,
        mean_age,
        SUM(seconds_viewed) as total_seconds_viewed, 
        MIN(program.program_total_duration) as program_total_duration,
        CASE 
            # IF viewed more than 1 hour
            # WHEN SUM(seconds_viewed) > 3600
            # IF viewed 75% of program length
            WHEN SUM(seconds_viewed) / MIN(program.program_total_duration) > 0.75
            # IF viewed more than 4 episodes
            OR SUM(seconds_viewed) > 4 * (MIN(program.program_total_duration) / AVG(count_episodes))
            THEN 1 
            ELSE 0 
        END AS count_viewed
    FROM {table} uv
    LEFT JOIN (
        # Duration of a program
        SELECT 
            program_id,
            COUNT(video_id) as count_episodes,
            AVG(mean_age) as mean_age,
            SUM(duration) as program_total_duration
        FROM (
            # Duration of each video
            SELECT 
                program_id,
                video_id,
                AVG(age) as mean_age,
                MAX(video_duration) as duration
            FROM {table}
            WHERE video_format IN ('preview', 'svod', 'freevod', 'replay')
            GROUP BY program_id, video_id
        )
        GROUP BY program_id
        ORDER BY program_total_duration desc
    ) program ON program.program_id  = uv.program_id
    WHERE video_format IN ('preview', 'svod', 'freevod', 'replay')
    GROUP BY pid, program_id, mean_age, gender, profile_type
    HAVING total_seconds_viewed > 120
) stats
LEFT JOIN (
    SELECT 
        program_id, 
        MIN(program_title) as title,
        tag_genre, 
        tag_sous_genre, 
        program_description
    FROM {table}
    GROUP BY program_id, tag_genre, tag_sous_genre, program_description
) program ON program.program_id = stats.program_id
GROUP BY program.program_id, program.title, tag_genre, tag_sous_genre, mean_age, count_episodes, program_description
HAVING count_profile_view >= 50
ORDER BY count_profile_like desc
"""
dataset = query_db(DATASET)
dataset['ratio_liked'] = round((dataset.count_profile_like / dataset.count_profile_view) * 100, 2)

# Separating each sous_genre
def get_sous_genre(sous_genre, index):
    if not sous_genre:
        return None
    sous_genre_split = sous_genre.split(',')
    sous_genre_1 = sous_genre_split[0]
    sous_genre_2 = sous_genre_split[1] if len(sous_genre_split) == 2 else None
    
    return sous_genre_1 if index == 1 else sous_genre_2

dataset['tag_sous_genre_1'] = dataset.tag_sous_genre.apply(lambda x: get_sous_genre(x, 1))
dataset['tag_sous_genre_2'] = dataset.tag_sous_genre.apply(lambda x: get_sous_genre(x, 2))
dataset = dataset.drop(['tag_sous_genre'], axis=1)

dataset.to_csv('dataset_v4.csv')
dataset

,program_id,title,tag_genre,mean_age,count_episodes,mean_episode_duration,count_profile_adult,count_profile_kid,count_profile_home,count_profile_f,...,count_profile_not_like,count_profile_view,program_seconds_duration,total_seconds_viewed,seconds_viewed_liked,seconds_viewed_not_like,program_description,ratio_liked,tag_sous_genre_1,tag_sous_genre_2
0,50753,Les Marseillais,divertissement,26.0,51.0,2633.29,120665,7,8145,80654,...,31591,129519,134298.0,5.800854e+09,5.655517e+09,145336850.0,"Pour cette nouvelle saison, la grande famille ...",75.61,telerealite,None
1,50636,La Villa des Cœurs Brisés,divertissement,27.0,56.0,2681.75,71056,4,4313,51058,...,18957,75808,150178.0,3.652003e+09,3.568976e+09,83026593.0,"Depuis 5 saisons déjà, des cœurs brisés s’envo...",74.99,telerealite,None
2,47981,Ici tout commence,series,35.0,65.0,1689.69,65259,3,3651,44706,...,13164,69302,109830.0,2.764460e+09,2.723784e+09,40676962.0,L’institut Auguste Armand est l’une des meille...,81.00,feuilleton,None
3,46079,Clem,series,31.0,12.0,3207.17,61041,1,2353,46238,...,25626,63512,38486.0,8.812518e+08,7.220255e+08,159226317.0,Clem se réveille après avoir passé 6 ans dans ...,59.65,comedie-dramatique,None
4,49623,Demain nous appartient,series,37.0,65.0,1818.31,45028,1,2601,31267,...,11521,47876,118190.0,1.787644e+09,1.750851e+09,36792547.0,"La série ""Demain nous appartient"" se déroule à...",75.94,feuilleton,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,43066,Décollage pour l'Amérique,documentaire,46.0,5.0,2966.60,73,0,2,29,...,75,75,14833.0,1.481720e+05,0.000000e+00,148172.0,Décollage pour l'Amérique,0.00,voyage-decouvertes,None
1588,42899,C politique,magazine,42.0,12.0,4703.75,54,0,1,21,...,55,55,56445.0,2.406650e+05,0.000000e+00,240665.0,"Karim Rissouli, entouré de Camille Girerd, Mer...",0.00,politique,None
1589,45818,Conférence de presse du premier ministre Jean ...,magazine,36.0,9.0,4113.44,227,0,5,146,...,232,232,37021.0,6.725650e+05,0.000000e+00,672565.0,"En raison de l’actualité, TF1 a décidé de modi...",0.00,actualite-info,None
1590,42721,"13h15, le samedi...",infos,47.0,9.0,1864.11,54,0,2,33,...,56,56,16777.0,9.516700e+04,0.000000e+00,95167.0,"Ce magazine donne un regard sur l'actualité, l...",0.00,actualite-info,None


In [4]:
# Transforming all letters to lowercase and removing missing values from program_description feature
stripped_corpus = [text.strip() for text in dataset.program_description.dropna()]

# Join corpus to make one full string
join_corpus = " ".join(stripped_corpus)

# Decode text 
clean_corpus = join_corpus.strip()

doc = nlp(clean_corpus)


In [5]:
# Removing stopwords and keep only letters
lemma = [token.lemma_ for token in doc if token.lemma_ not in STOP_WORDS and token.is_alpha]

# Create a DataFrame of lemmatized token
lemmadf = pd.Series(lemma)

# Create a mask of all counted values greater than 22
common = lemmadf.value_counts()

# 20 first common words
common[:20]

faire        349
aller        323
vie          300
an           249
jeune        225
nouveau      223
petit        204
grand        199
femme        191
famille      179
découvrir    173
devoir       172
monde        166
ami          158
enfant       155
histoire     154
homme        143
retrouver    139
jour         128
pouvoir      125
dtype: int64

In [6]:
# updating stopwords with the common words (removing them from our list)
new_stop_words = common[:5].index

# Create a union of sets 
STOP_WORDS_updated = STOP_WORDS.union(new_stop_words)

In [7]:
lemma[:20]

['nouveau',
 'saison',
 'grand',
 'famille',
 'marseillais',
 'retrouve',
 'Dubaï',
 'forme',
 'jamais',
 'véritable',
 'mirage',
 'cœur',
 'désert',
 'plage',
 'rêve',
 'paysage',
 'nuit',
 'Dubaï',
 'magnifique',
 'finir']

In [8]:
# Creating lemmatization
lemma = [token.lemma_ for token in doc if token.lemma_ not in STOP_WORDS_updated and token.is_alpha]

# Creating a new corpus 
corpus_lemma = " ".join(lemma)

corpus_series = pd.Series(corpus_lemma)

### Creating TF-IDF Matrix

In [9]:
# Defining a TF-IDF Vectorizer Object and Removing all french stop words
vectorizer = TfidfVectorizer(stop_words=STOP_WORDS_updated, smooth_idf=True)
X = vectorizer.fit_transform(corpus_series)

# X is a generator. We can transform that as an array
dense = X.toarray()
dense

C:\Users\elena\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


array([[0.00093809, 0.00093809, 0.00093809, ..., 0.00375235, 0.00844279,
        0.00093809]])

In [10]:
# Creating a DataFrame with all the words and tf-idf score 
tfid_df = pd.DataFrame(dense, columns=[x for x in vectorizer.get_feature_names()])

tfid_df

,aaron,ab,abandon,abandonne,abandonner,abandonné,abandonnée,abat,abattit,abattoir,...,évoquer,évènement,événement,événementiel,êtres,île,ôte,œil,œuvre,œuvrer
0,0.000938,0.000938,0.000938,0.002814,0.009381,0.000938,0.000938,0.000938,0.000938,0.000938,...,0.002814,0.005629,0.018762,0.000938,0.000938,0.028143,0.000938,0.003752,0.008443,0.000938


In [11]:
# Transpose dataframe
tfid_df = tfid_df.T

# Create a new token column 
tfid_df["token"] = tfid_df.index

tfid_df

,0,token
aaron,0.000938,aaron
ab,0.000938,ab
abandon,0.000938,abandon
abandonne,0.002814,abandonne
abandonner,0.009381,abandonner
...,...,...
île,0.028143,île
ôte,0.000938,ôte
œil,0.003752,œil
œuvre,0.008443,œuvre


In [12]:
# Sorting the top 15 values by descending order
tfid_df.sort_values(by=0, ascending=False)[:10]

,0,token
nouveau,0.210132,nouveau
petit,0.191370,petit
grand,0.186679,grand
femme,0.179175,femme
famille,0.167918,famille
monde,0.166980,monde
découvrir,0.162289,découvrir
devoir,0.161351,devoir
ami,0.148218,ami
enfant,0.145404,enfant


In [14]:
# Transforming all letters to lowercase
dataset["cleaned_description"] = dataset["program_description"].str.lower()

# Removing stopwords
dataset["cleaned_description"] = dataset["cleaned_description"].apply(lambda x: [token.lemma_ for token in nlp(x) if token.lemma_ not in STOP_WORDS_updated and token.is_alpha] if x else '')

# Join text
dataset["cleaned_description"] = dataset["cleaned_description"].apply(lambda x: " ".join(x))

## Creating Model

In [15]:
# TF-IDF vector
tfidf = TfidfVectorizer()

# Constructing the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(dataset['cleaned_description'])

# Outputing the shape of tfidf_matrix
tfidf_matrix.shape

(1592, 10212)

From the above output, we observe that 10212 different vocabularies of words in our dataset from 1592 programs.

In [16]:
# Array mapping from feature integer indices to feature name.
tfidf.get_feature_names()[3000:3010]

['démoniaque',
 'démontre',
 'démontrer',
 'démuni',
 'démunie',
 'déménagement',
 'déménager',
 'déménageur',
 'démêle',
 'démêler']

In [17]:
# Matrix of all TF-IDF words
dense = tfidf_matrix.todense()

# Transforming the TF-IDF dense to dataframe
df_tfidf = pd.DataFrame(dense, columns=[x for x in tfidf.get_feature_names()])
df_tfidf

,aaron,ab,abandon,abandonne,abandonner,abandonné,abandonnée,abat,abattit,abattoir,...,évoquer,évènement,événement,événementiel,êtres,île,ôte,œil,œuvre,œuvrer
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1588,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1590,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# Saving TF-IDF dataframe
df_tfidf.to_csv('tfidf_v4.csv', index=False)

In [19]:
def get_similar_description(program):
    
    # Concating dataset with df-idf dataframe
    tfidf_dataset = pd.concat([dataset, df_tfidf], axis=1)
    
    #index_value = tfidf_dataset[tfidf_dataset.program_id == program.program_id].index.values[0]
    results = cosine_similarity([df_tfidf.loc[program.name]], df_tfidf)
    tfidf_dataset['cos_similarity'] = results[0]

    return tfidf_dataset[['program_id', 'title', 'program_description', 'cos_similarity']].sort_values(by='cos_similarity', ascending=False)

In [20]:
def get_random_program():
    program = dataset.sample(1).iloc[0]
    print('titre: {}'.format(program.title))
    print('genre: {}'.format(program.tag_genre))
    print('sous-genre 1: {}'.format(program.tag_sous_genre_1))
    print('sous-genre 2: {}'.format(program.tag_sous_genre_2))
    return program
random_program = get_random_program()

titre: Très très bon
genre: magazine
sous-genre 1: cuisine
sous-genre 2: None


In [21]:
random = get_random_program()
similar_programs = get_similar_description(random).reset_index()
similar_programs.head(10)

titre: Profilage
genre: series
sous-genre 1: policier
sous-genre 2: None


,index,program_id,title,program_description,cos_similarity
0,1567,44412,Profilage,Le Commandant Rocher (Philippe Bas) est de ret...,1.000000
1,899,50134,Police Criminelle 24h/24,"Lorsqu’un homicide est commis, les équipes d’e...",0.117620
2,88,47837,Astrid et Raphaëlle,Lorsque le commandant Raphaëlle Coste s'adress...,0.109119
3,1299,51677,L'arme fatale 3,"Après un faux pas regrettable, Riggs et Murtau...",0.099956
4,503,47518,Où es-tu ?,Philippe et Suzanne se connaissent et s'aiment...,0.096870
5,1005,49186,Tropiques criminels,"Mélissa, excellente enquêtrice d'origine marti...",0.090777
6,214,46915,Alex Hugo,"Grande figure de la police marseillaise, Alex ...",0.089935
7,197,46779,Le prix de la vérité,"À Saint-Paul-de-Vence, on retrouve le cadavre ...",0.084829
8,1200,43511,Londres Police Judiciaire,Il s'agit de la résolution de crimes au fur et...,0.080785
9,275,43456,Cerise,"A 14 ans, Cerise est incontrôlable. Sa mère dé...",0.078087


In [22]:
pblv = dataset.iloc[9]
similar_programs = get_similar_description(pblv)
similar_programs.head(10)

,program_id,title,program_description,cos_similarity
9,47058,Plus belle la vie,"Derrière son comptoir, Roland Marci, tenancier...",1.000000
201,42775,Plus belle la vie,"A Marseille, dans le quartier du Mistral, tout...",0.936169
91,47438,Crimes parfaits,Des meurtriers pensent avoir commis le crime p...,0.116903
953,44364,Monde des Titounis,"Bienvenue dans le Monde des Titounis ! Ici, le...",0.103500
1035,42872,Masha et Michka,Masha est une petite fille particulièrement cu...,0.092447
730,45596,Les aventures de Petit Ours Brun,"Oh oh tiens, voilà Petit Ours Brun ! Dans un u...",0.083154
901,48989,Perfect Life,"Maria, Esther et Cristina, trois femmes au car...",0.080643
1051,45514,Miffy,Miffy est une charmante petite lapine qui vit ...,0.078980
1061,46214,Les blessures de l'île,Manon termine de longues études en neurochirur...,0.077325
1397,47639,Friends : Cinq filles en mission,Les filles vivent pleinement leurs passions et...,0.077246
